In [4]:
import os
import pandas as pd
import numpy as np
import requests
import warnings

from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
from pathlib import Path
import typing as tp
from tqdm import tqdm

from lightfm import LightFM

from implicit.bpr import BayesianPersonalizedRanking

from implicit.lmf import LogisticMatrixFactorization

In [3]:
url = "https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip"

req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [4]:
!unzip kion_train.zip

Archive:  kion_train.zip
   creating: kion_train/
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


In [5]:
warnings.filterwarnings('ignore')

In [6]:
os.environ["OPENBLAS_NUM_THREADS"] = "1" 

# Load data

In [7]:
interactions = pd.read_csv('kion_train/interactions.csv')
users = pd.read_csv('kion_train/users.csv')
items = pd.read_csv('kion_train/items.csv')

# Users

In [8]:
users.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [9]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840197 entries, 0 to 840196
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   840197 non-null  int64 
 1   age       826102 non-null  object
 2   income    825421 non-null  object
 3   sex       826366 non-null  object
 4   kids_flg  840197 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 32.1+ MB


# Items

In [10]:
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [11]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15963 entries, 0 to 15962
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       15963 non-null  int64  
 1   content_type  15963 non-null  object 
 2   title         15963 non-null  object 
 3   title_orig    11218 non-null  object 
 4   release_year  15865 non-null  float64
 5   genres        15963 non-null  object 
 6   countries     15926 non-null  object 
 7   for_kids      566 non-null    float64
 8   age_rating    15961 non-null  float64
 9   studios       1065 non-null   object 
 10  directors     14454 non-null  object 
 11  actors        13344 non-null  object 
 12  description   15961 non-null  object 
 13  keywords      15540 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 1.7+ MB


# Interactions

In [12]:
interactions.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [13]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   user_id        int64  
 1   item_id        int64  
 2   last_watch_dt  object 
 3   total_dur      int64  
 4   watched_pct    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 208.9+ MB


# Preprocess

In [14]:
Columns.Datetime = 'last_watch_dt'
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')   #bringing the date to a single format

In [15]:
max_date = interactions[Columns.Datetime].max()
print("Max date of interactions: ", max_date)

Max date of interactions:  2021-08-22 00:00:00


In [16]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)   #if the user viewed more than 10%, then the weight of interactions = 3, otherwise 1

In [17]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()   #data separation, test - data about the last week 

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [18]:
cold_users = set(test[Columns.User]) - set(train[Columns.User])   #filter out cold users from the test
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Prepare features

## User features

In [19]:
users.isnull().sum()

user_id         0
age         14095
income      14776
sex         13831
kids_flg        0
dtype: int64

In [20]:
users.fillna('Unknown', inplace=True)

In [21]:
users.nunique()

user_id     840197
age              7
income           7
sex              3
kids_flg         2
dtype: int64

In [22]:
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [23]:
user_features_frames = []
for feature in ["sex", "age"]:    #as features, we take the sex  and age of the user
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


In [24]:
user_features.query(f"id == 973171")

,id,value,feature
0,973171,М,sex
0,973171,age_25_34,age


## Item features

In [25]:
items.isnull().sum()

item_id             0
content_type        0
title               0
title_orig       4745
release_year       98
genres              0
countries          37
for_kids        15397
age_rating          2
studios         14898
directors        1509
actors           2619
description         2
keywords          423
dtype: int64

In [26]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

In [27]:
items.nunique()

item_id         15565
content_type        2
title           14937
title_orig      10377
release_year      105
genres           2720
countries         676
for_kids            2
age_rating          6
studios            38
directors        7809
actors          12671
description     15225
keywords        15123
dtype: int64

## Genre

In [28]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")       #explode genres to flatten table
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


## Content

In [29]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
item_features = pd.concat((genre_feature, content_feature))

In [30]:
item_features

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


# Metrics

In [31]:
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

# Models

In [32]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 8
N_FACTORS = (20, 30)
N_EPOCHS = (6, ) 
USER_ALPHA = [0, 0.1] 
ITEM_ALPHA = [0, 0.1] 
LEARNING_RATE = 0.05 

In [33]:
models = {
    'popular': PopularModel(),
}

In [34]:
implicit_models = {
    'ALS': AlternatingLeastSquares,
}

for implicit_name, implicit_model in implicit_models.items():
    for is_fitting_features in (True, False):
        for n_factors in N_FACTORS:
            print(n_factors)
            models[f"{implicit_name}_n_factors:{n_factors}_is_fitting_features:{is_fitting_features}"] = (
                ImplicitALSWrapperModel(
                    model=implicit_model(
                        factors=n_factors, 
                        random_state=RANDOM_STATE, 
                        num_threads=NUM_THREADS,
                    ),
                    fit_features_together=is_fitting_features,
                )
            )

20
30
20
30


In [35]:
lightfm_losses = ('bpr', 'warp') 

for n_epoch in N_EPOCHS:
  for user_alpha in USER_ALPHA:
    for item_alpha in ITEM_ALPHA:
      for loss in lightfm_losses:
          for n_factors in N_FACTORS:
              models[f"LightFM_{loss}_n_factors:{n_factors}_user_alpha:{user_alpha}_item_alpha:{item_alpha}_n_epoch:{n_epoch}"] = LightFMWrapperModel(
                  LightFM(
                      no_components=n_factors, 
                      loss=loss, 
                      random_state=RANDOM_STATE,
                      learning_rate=LEARNING_RATE,
                      user_alpha=user_alpha,
                      item_alpha=item_alpha,
                  ),
                  epochs=n_epoch,
                  num_threads=NUM_THREADS,
              )

In [36]:
%%time
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

CPU times: user 1.94 s, sys: 33.3 ms, total: 1.97 s
Wall time: 2.05 s


In [37]:
test_users = test[Columns.User].unique()

In [38]:
%%time
results = []
for model_name, model in models.items():
    print(f"Fitting model {model_name}...")
    model_quality = {'model': model_name}

    model.fit(dataset)
    recos = model.recommend(
        users=test_users,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    metric_values = calc_metrics(metrics, recos, test, train)
    model_quality.update(metric_values)
    results.append(model_quality)

Fitting model popular...
Fitting model ALS_n_factors:20_is_fitting_features:True...
Fitting model ALS_n_factors:30_is_fitting_features:True...
Fitting model ALS_n_factors:20_is_fitting_features:False...
Fitting model ALS_n_factors:30_is_fitting_features:False...
Fitting model LightFM_bpr_n_factors:20_user_alpha:0_item_alpha:0_n_epoch:6...
Fitting model LightFM_bpr_n_factors:30_user_alpha:0_item_alpha:0_n_epoch:6...
Fitting model LightFM_warp_n_factors:20_user_alpha:0_item_alpha:0_n_epoch:6...
Fitting model LightFM_warp_n_factors:30_user_alpha:0_item_alpha:0_n_epoch:6...
Fitting model LightFM_bpr_n_factors:20_user_alpha:0_item_alpha:0.1_n_epoch:6...
Fitting model LightFM_bpr_n_factors:30_user_alpha:0_item_alpha:0.1_n_epoch:6...
Fitting model LightFM_warp_n_factors:20_user_alpha:0_item_alpha:0.1_n_epoch:6...
Fitting model LightFM_warp_n_factors:30_user_alpha:0_item_alpha:0.1_n_epoch:6...
Fitting model LightFM_bpr_n_factors:20_user_alpha:0.1_item_alpha:0_n_epoch:6...
Fitting model LightFM

In [39]:
df_quality = pd.DataFrame(results).T

df_quality.columns = df_quality.iloc[0]

df_quality.drop('model', inplace=True)

In [40]:
df_quality.style.highlight_max(color='lightgreen', axis=1)

model,popular,ALS_n_factors:20_is_fitting_features:True,ALS_n_factors:30_is_fitting_features:True,ALS_n_factors:20_is_fitting_features:False,ALS_n_factors:30_is_fitting_features:False,LightFM_bpr_n_factors:20_user_alpha:0_item_alpha:0_n_epoch:6,LightFM_bpr_n_factors:30_user_alpha:0_item_alpha:0_n_epoch:6,LightFM_warp_n_factors:20_user_alpha:0_item_alpha:0_n_epoch:6,LightFM_warp_n_factors:30_user_alpha:0_item_alpha:0_n_epoch:6,LightFM_bpr_n_factors:20_user_alpha:0_item_alpha:0.1_n_epoch:6,LightFM_bpr_n_factors:30_user_alpha:0_item_alpha:0.1_n_epoch:6,LightFM_warp_n_factors:20_user_alpha:0_item_alpha:0.1_n_epoch:6,LightFM_warp_n_factors:30_user_alpha:0_item_alpha:0.1_n_epoch:6,LightFM_bpr_n_factors:20_user_alpha:0.1_item_alpha:0_n_epoch:6,LightFM_bpr_n_factors:30_user_alpha:0.1_item_alpha:0_n_epoch:6,LightFM_warp_n_factors:20_user_alpha:0.1_item_alpha:0_n_epoch:6,LightFM_warp_n_factors:30_user_alpha:0.1_item_alpha:0_n_epoch:6,LightFM_bpr_n_factors:20_user_alpha:0.1_item_alpha:0.1_n_epoch:6,LightFM_bpr_n_factors:30_user_alpha:0.1_item_alpha:0.1_n_epoch:6,LightFM_warp_n_factors:20_user_alpha:0.1_item_alpha:0.1_n_epoch:6,LightFM_warp_n_factors:30_user_alpha:0.1_item_alpha:0.1_n_epoch:6
Precision@1,0.073308,0.085862,0.085862,0.062164,0.062164,0.025100,0.025091,0.080527,0.079763,0.000000,0.000008,0.000008,0.000000,0.001668,0.001742,0.000008,0.000000,0.000000,0.000000,0.078552,0.001784
Recall@1,0.038149,0.044848,0.044848,0.031754,0.031754,0.013332,0.012894,0.040703,0.040429,0.000000,0.000000,0.000004,0.000000,0.000794,0.000934,0.000000,0.000000,0.000000,0.000000,0.041690,0.000573
Precision@2,0.069263,0.073258,0.073258,0.055311,0.055311,0.019603,0.019806,0.068118,0.067657,0.000000,0.000004,0.000004,0.000004,0.001253,0.003066,0.000004,0.000000,0.000004,0.000000,0.069728,0.024498
Recall@2,0.071011,0.074754,0.074754,0.055191,0.055191,0.020478,0.019958,0.067612,0.067233,0.000000,0.000000,0.000004,0.000003,0.001127,0.003515,0.000000,0.000000,0.000000,0.000000,0.071757,0.025111
Precision@3,0.066225,0.062087,0.062087,0.051309,0.051309,0.016174,0.016681,0.059836,0.059490,0.000000,0.000008,0.000017,0.000003,0.001048,0.003156,0.000008,0.000000,0.000003,0.000000,0.065882,0.016407
Recall@3,0.100400,0.093394,0.093394,0.075896,0.075896,0.024957,0.024634,0.087931,0.087446,0.000000,0.000002,0.000011,0.000003,0.001359,0.005431,0.000001,0.000000,0.000000,0.000000,0.100393,0.025192
Precision@4,0.059383,0.054610,0.054610,0.047447,0.047447,0.014052,0.014637,0.053664,0.053351,0.000004,0.000012,0.000015,0.000012,0.000919,0.003205,0.000008,0.000006,0.000002,0.000000,0.057167,0.012593
Recall@4,0.118878,0.108308,0.108308,0.092287,0.092287,0.028310,0.028330,0.104015,0.103159,0.000001,0.000005,0.000011,0.000015,0.001571,0.007293,0.000009,0.000001,0.000000,0.000000,0.114564,0.025467
Precision@5,0.052735,0.048517,0.048517,0.043812,0.043812,0.012635,0.013160,0.048819,0.048215,0.000015,0.000018,0.000015,0.000012,0.001271,0.003123,0.000010,0.000022,0.000002,0.000000,0.052038,0.010320
Recall@5,0.130473,0.118726,0.118726,0.105005,0.105005,0.031346,0.031610,0.117217,0.115453,0.000020,0.000011,0.000022,0.000017,0.002867,0.008820,0.000019,0.000039,0.000000,0.000000,0.129007,0.025837
